# MusciVAE

- input: 130차원(velocity) -> T, 9
- drum class: 9 canonical classes
- 16개의 note intervals -> 16 events
- 2-bar data -> T=32, 16-bar data -> T=256
- U = 16 -> 1개의 bar 안에 16개

- 2-bar
- input: T, 130, 9
- bar별로 16개의 note들
- rnn model: slide 1bar
- output: U=16

In [ ]:
"""
    Create a pandas to store the piano rolls in.
    It'll look a bit like this:
    
    |   Index     | t | C0 |...| C10 |
    |Song_name_3:0| 0 | 40 |...| 0   |
    |             | 1 | 40 |...| 0   |
    |             |...|... |...| ... |
    |             | N | 40 |...| 0   |
    |Song_name_3:1| 0 | 40 |...| 0   |
    |             |...|... |...| ... |
    |             | N | 40 |...| 0   |
    |Song_name_4:0| 0 | 40 |...| 0   |
    |             |...|... |...| ... |
    |             | N | 40 |...| 0   |

"""

In [4]:
import pandas as pd
import pretty_midi
import os
from getpass import getuser
import numpy as np

# EDA

In [2]:
username = getuser()
base_path = f'/Users/{username}/data/midi/groove'
info_csv_filename = os.path.join(base_path, 'info.csv')

info_df = pd.read_csv(info_csv_filename)
info_df

,drummer,session,id,style,bpm,beat_type,time_signature,midi_filename,audio_filename,duration,split
0,drummer1,drummer1/eval_session,drummer1/eval_session/1,funk/groove1,138,beat,4-4,drummer1/eval_session/1_funk-groove1_138_beat_...,drummer1/eval_session/1_funk-groove1_138_beat_...,27.872308,test
1,drummer1,drummer1/eval_session,drummer1/eval_session/10,soul/groove10,102,beat,4-4,drummer1/eval_session/10_soul-groove10_102_bea...,drummer1/eval_session/10_soul-groove10_102_bea...,37.691158,test
2,drummer1,drummer1/eval_session,drummer1/eval_session/2,funk/groove2,105,beat,4-4,drummer1/eval_session/2_funk-groove2_105_beat_...,drummer1/eval_session/2_funk-groove2_105_beat_...,36.351218,test
3,drummer1,drummer1/eval_session,drummer1/eval_session/3,soul/groove3,86,beat,4-4,drummer1/eval_session/3_soul-groove3_86_beat_4...,drummer1/eval_session/3_soul-groove3_86_beat_4...,44.716543,test
4,drummer1,drummer1/eval_session,drummer1/eval_session/4,soul/groove4,80,beat,4-4,drummer1/eval_session/4_soul-groove4_80_beat_4...,drummer1/eval_session/4_soul-groove4_80_beat_4...,47.987500,test
...,...,...,...,...,...,...,...,...,...,...,...
1145,drummer2,drummer2/session2,drummer2/session2/11,rock,130,beat,4-4,drummer2/session2/11_rock_130_beat_4-4.mid,NaN,1.909613,train
1146,drummer2,drummer2/session2,drummer2/session2/12,rock,130,beat,4-4,drummer2/session2/12_rock_130_beat_4-4.mid,NaN,1.808652,train
1147,drummer2,drummer2/session2,drummer2/session2/13,rock,130,beat,4-4,drummer2/session2/13_rock_130_beat_4-4.mid,NaN,1.864421,train
1148,drummer2,drummer2/session2,drummer2/session2/14,rock,130,beat,4-4,drummer2/session2/14_rock_130_beat_4-4.mid,NaN,1.875960,train


In [9]:
sample_filename = info_df[info_df.split.eq('train')]['midi_filename'].iloc[0]
sample_filename = os.path.join(base_path, sample_filename)

midi = pretty_midi.PrettyMIDI(sample_filename)
ns = midi.instruments[0]

In [9]:
midi = pretty_midi.PrettyMIDI(sample_filename)
ns = midi.instruments[0]

In [5]:
ROLAND_DRUM_PITCH_CLASSES = [
    # kick drum
    [36],
    # snare drum
    [38, 37, 40],
    # closed hi-hat
    [42, 22, 44],
    # open hi-hat
    [46, 26],
    # low tom
    [43, 58],
    # mid tom
    [47, 45],
    # high tom
    [50, 48],
    # crash cymbal
    [49, 52, 55, 57],
    # ride cymbal
    [51, 53, 59]
]

def _classes_to_map(classes):
      class_map = {}
      for cls, pitches in enumerate(classes):
        for pitch in pitches:
          class_map[pitch] = cls
      return class_map

def class_map():
    return _classes_to_map(ROLAND_DRUM_PITCH_CLASSES)

"""
    Create a pandas to store the piano rolls in.
    It'll look a bit like this:
    
    |   Index     | t | C0 |...| C10 |
    |Song_name_3:0| 0 | 40 |...| 0   |
    |             | 1 | 40 |...| 0   |
    |             |...|... |...| ... |
    |             | N | 40 |...| 0   |
    |Song_name_3:1| 0 | 40 |...| 0   |
    |             |...|... |...| ... |
    |             | N | 40 |...| 0   |
    |Song_name_4:0| 0 | 40 |...| 0   |
    |             |...|... |...| ... |
    |             | N | 40 |...| 0   |

"""

"\n    Create a pandas to store the piano rolls in.\n    It'll look a bit like this:\n    \n    |   Index     | t | C0 |...| C10 |\n    |Song_name_3:0| 0 | 40 |...| 0   |\n    |             | 1 | 40 |...| 0   |\n    |             |...|... |...| ... |\n    |             | N | 40 |...| 0   |\n    |Song_name_3:1| 0 | 40 |...| 0   |\n    |             |...|... |...| ... |\n    |             | N | 40 |...| 0   |\n    |Song_name_4:0| 0 | 40 |...| 0   |\n    |             |...|... |...| ... |\n    |             | N | 40 |...| 0   |\n\n"

In [6]:
len(ROLAND_DRUM_PITCH_CLASSES)
import numpy as np

midi = pretty_midi.PrettyMIDI(sample_filename)
ns = midi.instruments[0]

note_sequences = []
for i, note in enumerate(ns.notes):
    columns = np.zeros(len(ROLAND_DRUM_PITCH_CLASSES))
    columns[class_map()[note.pitch]] = note.velocity/127
    note_sequences.append(columns)

In [45]:
sample_filenames = info_df[info_df.split.eq('train')]['midi_filename']

note_dfs = []
for sample_filename in sample_filenames[:10]:
    sample_filename = os.path.join(base_path, sample_filename)
    midi = pretty_midi.PrettyMIDI(sample_filename)

    ns = midi.instruments[0]
    note_sequences = []
    for i, note in enumerate(ns.notes):
        columns = np.zeros(len(ROLAND_DRUM_PITCH_CLASSES))
        columns[class_map()[note.pitch]] = note.velocity/127
        note_sequences.append(columns)

    note_seq_df = pd.DataFrame(note_sequences)
    note_seq_df.reset_index(inplace=True)
    note_seq_df['filename'] = os.path.basename(sample_filename).split('.')[0]
    note_seq_df.rename(columns={'index': 'timestep'}, inplace=True)
    note_dfs.append(note_seq_df)

/var/folders/v0/x10gdwv977v5nmwzt4vzq50w0000gn/T/ipykernel_96113/1016426541.py:4: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  for sample_filename in sample_filenames[:10]:


In [46]:
drum_df = pd.concat(note_dfs, axis=0)
drum_df.set_index(['filename', 'timestep'], inplace=True)

In [60]:
import numpy as np
import math

class BarTransform():

    def __init__(self, bars=1, note_count=60):
        self.split_size = bars*16
        self.note_count = note_count

    def get_sections(self, sample_length):
        return math.ceil(sample_length/ self.split_size)

    def __call__(self, sample):
        sample_length = len(sample)

        # Pad the sample with 0's if there's not enough to create equal splits into n bars
        leftover = sample_length % self.split_size
        if leftover != 0:
            padding_size = self.split_size - leftover
            padding = np.zeros((padding_size, self.note_count))
            sample = np.append(sample, padding, axis=0)


        sections = self.get_sections(sample_length)
        # Split into X equal sections
        split_list = np.array_split(sample, indices_or_sections=sections)


        return split_list

In [47]:
drum_df

0         1         2    3         4  \
filename             timestep                                                
1_funk_80_beat_4-4   0         0.000000  0.055118  0.000000  0.0  0.000000   
                     1         0.000000  0.078740  0.000000  0.0  0.000000   
                     2         0.000000  0.440945  0.000000  0.0  0.000000   
                     3         0.000000  0.000000  0.511811  0.0  0.000000   
                     4         0.000000  0.377953  0.000000  0.0  0.000000   
...                                 ...       ...       ...  ...       ...   
108_funk_95_fill_4-4 11        0.000000  0.472441  0.000000  0.0  0.000000   
                     12        0.000000  0.000000  0.000000  0.0  0.629921   
                     13        0.000000  0.000000  0.000000  0.0  1.000000   
                     14        0.000000  0.944882  0.000000  0.0  0.000000   
                     15        0.456693  0.000000  0.000000  0.0  0.000000   

                                 5    6    7    8  
filename             timestep                      
1_funk_80_beat_4-4   0         0.0  0.0  0.0  0.0  
                     1         0.0  0.0  0.0  0.0  
                     2         0.0  0.0  0.0  0.0  
                     3         0.0  0.0  0.0  0.0  
                     4         0.0  0.0  0.0  0.0  
...                            ...  ...  ...  ...  
108_funk_95_fill_4-4 11        0.0  0.0  0.0  0.0  
                     12        0.0  0.0  0.0  0.0  
                     13        0.0  0.0  0.0  0.0  
                     14        0.0  0.0  0.0  0.0  
                     15        0.0  0.0  0.0  0.0  

[6316 rows x 9 columns]

In [55]:
indexer = drum_df.index.get_level_values(0).unique()


In [64]:
index_mapper = []
transform = BarTransform(2)
for i in indexer:
    split_count = transform.get_sections(len(drum_df.loc[i].values))
    for j in range(0, split_count):
        index_mapper.append((i, j))

In [65]:
index_mapper

[('1_funk_80_beat_4-4', 0),
 ('1_funk_80_beat_4-4', 1),
 ('1_funk_80_beat_4-4', 2),
 ('1_funk_80_beat_4-4', 3),
 ('1_funk_80_beat_4-4', 4),
 ('1_funk_80_beat_4-4', 5),
 ('1_funk_80_beat_4-4', 6),
 ('1_funk_80_beat_4-4', 7),
 ('1_funk_80_beat_4-4', 8),
 ('1_funk_80_beat_4-4', 9),
 ('1_funk_80_beat_4-4', 10),
 ('1_funk_80_beat_4-4', 11),
 ('1_funk_80_beat_4-4', 12),
 ('1_funk_80_beat_4-4', 13),
 ('1_funk_80_beat_4-4', 14),
 ('1_funk_80_beat_4-4', 15),
 ('1_funk_80_beat_4-4', 16),
 ('1_funk_80_beat_4-4', 17),
 ('1_funk_80_beat_4-4', 18),
 ('1_funk_80_beat_4-4', 19),
 ('1_funk_80_beat_4-4', 20),
 ('1_funk_80_beat_4-4', 21),
 ('1_funk_80_beat_4-4', 22),
 ('1_funk_80_beat_4-4', 23),
 ('1_funk_80_beat_4-4', 24),
 ('10_jazz-funk_116_fill_4-4', 0),
 ('100_neworleans-secondline_94_beat_4-4', 0),
 ('100_neworleans-secondline_94_beat_4-4', 1),
 ('100_neworleans-secondline_94_beat_4-4', 2),
 ('100_neworleans-secondline_94_beat_4-4', 3),
 ('100_neworleans-secondline_94_beat_4-4', 4),
 ('100_neworlea

In [67]:
drum_df.shape[0]

6316

In [ ]:
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import pretty_midi

class MidiDataset(Dataset):
    """Pre-processed MIDI dataset."""

    def __init__(self, csv_file, transform, midi_start=48, midi_end=108):
        """
        Args:
            csv_file (string): Path to the csv file with piano rolls per song.
            transform (callable): Transform to be applied on a sample, is expected to implement "get_sections".
            midi_start (int): The first midi note in the dataset
            midi_end (int): The last midi note in the dataset
        """

        # dtypes = {'piano_roll_name': 'object', 'timestep': 'uint32'}
        # column_names = [pretty_midi.note_number_to_name(n) for n in range(midi_start, midi_end)]
        # for column in column_names:
        #     dtypes[column] = 'uint8'

        self.drum_df = pd.read_csv(csv_file, sep=';', index_col=['filename', 'timestep'])
        self.transform = transform

        self.init_dataset()

    def init_dataset(self):
        """
            Sets up an array containing a pd index (the song name) and the song section,
            ie. [("Song Name:1", 0), ("Song Name:1", 1), ("Song Name:1", 2)]
            for use in indexing a specific section
        """
        indexer = self._get_indexer()

        self.index_mapper = []
        for i in indexer:
            split_count = self.transform.get_sections(len(self.drum_df.loc[i].values))
            for j in range(0, split_count):
                self.index_mapper.append((i, j))


    def __len__(self):
        return len(self.index_mapper)

    def get_mem_usage(self):
        """
            Returns the memory usage in MB
        """
        return self.drum_df.memory_usage(deep=True).sum() / 1024**2

    def _get_indexer(self):
        """
            Get an indexer that treats each first level index as a sample.
        """
        return self.drum_df.index.get_level_values(0).unique()

    def __getitem__(self, idx):
        """
            Our frame is multi-index, so we're thinking each song is a single sample,
            and getting the individual bars is a transform of that sample?
        """
        song_name, section = self.index_mapper[idx]

        # # Add a column for silences
        # drums = self.drum_df.loc[song_name].values
        # silence_col = np.zeros((drums.shape[0], 1))
        # piano_rolls_with_silences = np.append(drums, silence_col, axis=1)

        # Transform the sample (including padding)
        # sample = piano_rolls_with_silences.astype('float')
        sample = self.transform(sample)[section]

        # Fill in 1's for the silent rows
        empty_rows = ~sample.any(axis=1)
        if len(sample[empty_rows]) > 0:
            sample[empty_rows, -1] = 1.

        sample = {'piano_rolls': sample}

        return sample

In [1]:
from torch.utils.data import Dataset

class GrooveDataset(Dataset):
    def __init__(self, info_csv_filename, midi_dir, group='train'):
        info_df = pd.read_csv(info_csv_filename)
        self.midi_filenames = info_df[info_df.split.eq(group)]['midi_filename']
        self.midi_dir = midi_dir

    def __len__(self):
        return len(self.midi_filenames)

    def __getitem__(self, idx, ):
        item_filename = self.midi_filenames.iloc[idx]
        midi_filename = os.path.join(self.midi_dir, item_filename)
        midi = pretty_midi.PrettyMIDI(midi_filename)
        ns = midi.instruments[0]

        item = []
        for note in ns.notes:
            item.append([note.end-note.start, note.pitch, note.velocity])
        return item

In [2]:
base_path = '/Users/rimiiii/data/midi/groove'
info_csv_filename = os.path.join(base_path, 'info.csv')

groove_dataset = GrooveDataset(info_csv_filename, base_path)
groove_dataset[0]

NameError: name 'pd' is not defined

In [14]:
import tensorflow as tf
import tensorflow_datasets as tfds

# tfds works in both Eager and Graph modes
# tf.enable_eager_execution()

# Load the full GMD with MIDI only (no audio) as a tf.data.Dataset
dataset = tfds.load(
    name="groove/full-midionly",
    split=tfds.Split.TRAIN,
    try_gcs=True)

# Build your input pipeline
dataset = dataset.shuffle(1024).batch(32).prefetch(
    tf.data.experimental.AUTOTUNE)
for features in dataset.take(1):
  # Access the features you are interested in
  midi, genre = features["midi"], features["style"]["primary"]

2022-12-05 23:34:24.715893: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".
2022-12-05 23:34:30.773410: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [30]:
print(dataset.batch(32))

<BatchDataset element_spec={'bpm': TensorSpec(shape=(None, None), dtype=tf.int32, name=None), 'drummer': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'id': TensorSpec(shape=(None, None), dtype=tf.string, name=None), 'midi': TensorSpec(shape=(None, None), dtype=tf.string, name=None), 'style': {'primary': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'secondary': TensorSpec(shape=(None, None), dtype=tf.string, name=None)}, 'time_signature': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'type': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}>
